In [1]:
import pandas as pd
import shutil
import os
import gcsfs
from calitp_data_analysis.tables import tbls
from siuba import _, collect, count, filter, show_query
from shared_utils.rt_dates import MONTH_DICT
from update_vars import NTD_MODES, NTD_TOS, YEAR, MONTH

#temp file path for testing
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/csuyat_folder/"

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

### what is the grain of this dataset?
metrics values (upt/vrm/vrh/crm/voms) of a specifc **tos**, of specific **mode**, for a speicifc **agency** on a specific **year/month**

# Test reading in `dim_monthly_ntd_ridership_with_adjustments` from warehouse

In [ ]:
monthly_ridership = (
    tbls.mart_ntd.dim_monthly_ntd_ridership_with_adjustments() >> collect()
)

In [ ]:
# reading in old monthly ridership parquet for comparison
old_march_2024_ridership = pd.read_parquet("gs://calitp-analytics-data/data-analyses/ntd/ntd_monthly_ridership_2024_March.parquet")

In [ ]:
# what is in this table?
# all states
display(
    monthly_ridership.shape,
    # monthly_ridership.info,
    monthly_ridership.head(3),
)

In [ ]:
print(monthly_ridership.dtypes)

In [ ]:
# are there NaN values in uza name? YES
monthly_ridership["uza_name"].isna().sum()

In [ ]:
# fill the NaN values with something
monthly_ridership["uza_name"].fillna("none", inplace=True)

In [ ]:
# double check for na
monthly_ridership["uza_name"].isna().sum()

In [ ]:
# can i now filter by uza CA, becaue there are zero NaN values? YES
uza_ca = monthly_ridership[monthly_ridership["uza_name"].str.contains(", CA")]
display(
    uza_ca.shape,
    uza_ca["uza_name"].value_counts()
)

In [ ]:
# where are the UTP values? the UTP page for this table used to be very wide
# look at a specific agency by ntd id, LA Metro == "9154"
# what is the dtype of ntd_id... its object, so use str.contrains
la_metro = monthly_ridership[monthly_ridership["ntd_id"] == "90154"]
display(la_metro.shape, uza_ca.shape)

In [ ]:
# filter la_metro data to only see current and previous year (2024 and 2023)

la_2324 = la_metro[la_metro["period_year"].isin(["2023", "2024"])]

In [ ]:
# does this 23/24 data match whats in the NTD report? YES
# confirm that data is now long instead of wide
la_2324[la_2324["period_year_month"] == "2024-03"][
    ["period_year_month", "mode", "tos", "upt"]
].sort_values(by="period_year_month", ascending=True)

# What to modify in the `produce_ntd_monthly_ridership_by_rtpa` function
step-by-step

In [3]:
# 1. reading in the data & renaming column
# intead of reading in the data via an excel doc by URL, intead will query the warehouse directly

#GOOD TO GO
full_upt = (
    tbls.mart_ntd.dim_monthly_ntd_ridership_with_adjustments() >> collect()
).rename(columns = {"mode_type_of_service_status": "Status"}
       )

In [4]:
#checking for recent data
full_upt[full_upt["period_year_month"].str.contains("2024-")]["period_year_month"].sort_values().unique()

array(['2024-01', '2024-02', '2024-03', '2024-04', '2024-05'],
      dtype=object)

In [ ]:
#1.1 setting `ntd_id` to str

# NTD_ID COL IS ALREADY OBJECT/STR DATA TYPE
# full_upt["ntd_id"] = full_upt["ntd_id"].astype(str)

In [ ]:
#2 filter df to get rid of na rows
# GOOD TO GO!
full_upt = full_upt[full_upt.agency.notna()].reset_index(drop=True)

In [ ]:
#3 new DF of just CA and not-blank agency names
# filter UZA name col for CA and not blank agency names.

#GOOD TO GO!
ca = full_upt[(full_upt["uza_name"].str.contains(", CA")) & 
            (full_upt.agency.notna())].reset_index(drop=True)

In [ ]:
#4 read in ntd_id-to-RTPA crosswalk
# same as before, no changes

#GOOD TO GO!
crosswalk = pd.read_csv(
        f"gs://calitp-analytics-data/data-analyses/ntd/ntd_id_rtpa_crosswalk.csv", 
        dtype = {"NTD ID": "str"}
    #have to rename NTD ID col to match the dim table
    ).rename(columns={"NTD ID": "ntd_id"})
    
crosswalk.columns

In [ ]:
#5 merge CA df and crosswalk

# GOOD TO GO
df = pd.merge(
        ca,
        # Merging on too many columns can create problems 
        # because csvs and dtypes aren't stable / consistent 
        # for NTD ID, Legacy NTD ID, and UZA
        crosswalk[["ntd_id", "RTPA"]],
        #change on = value to ntd_id
        on = "ntd_id",
        how = "left",
        indicator = True
    )
    
print(df._merge.value_counts())

In [ ]:
#6 flag to see if there are any unmerged rows 

# GOOD TO GO
if len(df[df._merge=="left_only"]) > 0:
        raise ValueError("There are unmerged rows to crosswalk")

In [ ]:
#7 reverses the MONTH_DICT 

# Add new columns    
reversed_months = {v:k for k, v in MONTH_DICT.items()}
    
for m in range(1, reversed_months[month] + 1):
    df = add_change_columns(df, year, m)

In [ ]:
# FINAL FORM


# final updated  `produce_ntd_monthly_ridership_by_rtpa` function.
# may not need the upt_url

def produce_ntd_monthly_ridership_by_rtpa(
    #upt_url: str,
    year: int,
    month: str
) -> pd.DataFrame:
    """
    Import NTD data from url, filter to CA, 
    merge in crosswalk, and save individual csvs.
    """
    full_upt = (tbls.mart_ntd.dim_monthly_ntd_ridership_with_adjustments() >> collect()
               ).rename(columns = {"mode_type_of_service_status": "Status"})
    
    full_upt = full_upt[full_upt.agency.notna()].reset_index(drop=True)
    #full_upt.to_parquet(
    #    f"{GCS_FILE_PATH}ntd_monthly_ridership_{year}_{month}.parquet"
    #)
    ca = full_upt[(full_upt["uza_name"].str.contains(", CA")) & 
            (full_upt.agency.notna())].reset_index(drop=True)
    
    crosswalk = pd.read_csv(
        f"gs://calitp-analytics-data/data-analyses/ntd/ntd_id_rtpa_crosswalk.csv", 
        dtype = {"NTD ID": "str"}
    #have to rename NTD ID col to match the dim table
    ).rename(columns={"NTD ID": "ntd_id"})
    
    df = pd.merge(
        ca,
        # Merging on too many columns can create problems 
        # because csvs and dtypes aren't stable / consistent 
        # for NTD ID, Legacy NTD ID, and UZA
        crosswalk[["ntd_id", "RTPA"]],
        #change on = value to ntd_id
        on = "ntd_id",
        how = "left",
        indicator = True
    )
    
    print(df._merge.value_counts())
    
    # Good, everything merged, as we want
    if len(df[df._merge=="left_only"]) > 0:
        raise ValueError("There are unmerged rows to crosswalk")
        
    # Add new columns    
    reversed_months = {v:k for k, v in MONTH_DICT.items()}
    
    for m in range(1, reversed_months[month] + 1):
        df = add_change_columns(df, year, m)
    
    df = df.assign(
        Mode_full = df.Mode.map(NTD_MODES),
        TOS_full = df.TOS.map(NTD_TOS)
    )
    
    return df


---
## Do I need `reserved_months` anymore?
RE: `produce_ntd_monthly_ridership_by_rtpa` function.


Since the new table uses a `yyyy-mm` format now. i dont think i need the revesre months anymore. what else needs to change?
reversed month was needed for the `add_change_columns` function.  

**NO LONGER NEEDED SINCE `add_change_columns_v2` WORKS WITHOUT USING MONTH_DICT


In [ ]:
# what does reversed months do?

reversed_months = {v:k for k, v in MONTH_DICT.items()}

# what MONTH_DICT looks like initially.

print("this is MONTH_DICT"),
display(MONTH_DICT),

# like the name implies, reverses the key values pairs. 
print("this is reversed_months"),
display(reversed_months)


In [ ]:
# what is the date format in the new df
# YYYY-MM
df["period_year_month"].sample(3)

In [ ]:
# what is the date format in the old data?
# M/YYYY
old_march_2024_ridership.iloc[:,18:22].head()

## options
* change the df date format to match reversed_months format. (yyyy-mm to mm/yyy)
    * can use a mix of split, str replace, and reversing via slicing 
* change the `add_change_column` function to accept yyyy-mm
    * make a v2 of this function that accepts yyyy-mm, creates a new column, and applys some function by looking at the row period_year_month and finding the "period_year_month" -1 yr to calculate the difference.

# TEST update `add_change_columns` function
need to update this function so it can work with the `produce_ntd_monthly_ridership_by_rtpa` function

In [ ]:
# updated to match new year-month format 

def add_change_columns_v2(
    df: pd.DataFrame,
    year: int,
    month: int
) -> pd.DataFrame:
    """
    updated to accept yyyy-mm format.
    
    """    
    ntd_month_col = f"{year}-{month}"
    prior_year_col = f"{int(year)-1}-{month}"
        
    df[f"change_1yr_{ntd_month_col}"] = df[ntd_month_col] - df[prior_year_col]
    df = get_percent_change(df, ntd_month_col, prior_year_col)
    
    return df

In [ ]:
# can i use a lambda function to create a new column that:
# checks every row (x), if the row has a `period_year_month` value, does it have a `period_year_month` -1 year value in the same column? 
    # if true, then subtract `period_year_month` by `period_year_month`-1 year.
    # else return 0
year = 2023
month = "05"

current_y_m = f"{year}-{month}"
prior_y_m = f"{int(year)-1}-{month}"

# using current_y_m works and prior_year_col works
df[df["period_year_month"].str.contains(prior_y_m)].head(3)



In [ ]:
# first attempt at what im trying to do

# create a new column called "change 1yr"
# at each row in "period_month_year", do what? take the value of that row, then mimus the value of the previous period_mont_year -1

#this doesnt work becaues im not indicating how X is used, 
#df["change_1yr"] = df["period_year_month"].apply(lambda x: (df["period_year_month"].str.contains(current_y_m)["upt"]) - (df["period_year_month"].str.contains(prior_year_col)["upt"]))

#pseudo code
df["change_1yr"] = df["period_year_month"].apply(lambda x: (upt at x for current_y_m, agency, tos and mode)  - 
                                                 (upt at x for prior_y_m, current_y_m, agency, tos and mode))



In [ ]:
# how can i see the specific UPT for a specific agency, mode & TOS and date?

#uses .loc at current_y_m UPT, position 0, minus .loc at previous_year_col UPT, position o
# but is is the same agency/mode/tos in both sides of the equation?
df.loc[df["period_year_month"] == current_y_m,"upt"].values[0] - df.loc[df["period_year_month"] == prior_y_m,"upt"].values[0]

In [ ]:
df.loc[
        (df["period_year_month"] == current_y_m) &
        (df["ntd_id"] == df["ntd_id"]) &
        (df["mode"] == df["mode"]) &
        (df["tos"] == df["tos"]), "upt"
    ].head()

In [ ]:
df.sample(3)

In [ ]:
df.loc[df["period_year_month"] == prior_y_m, "upt"][:10]


In [ ]:
def test_difference_func(row, current_y_m, prior_y_m ):
    current_ym_val =  df.loc[
        (df["period_year_month"] == current_y_m) &
        (df["ntd_id"] == row["ntd_id"]) &
        (df["mode"] == row["mode"]) &
        (df["tos"] == row["tos"]), "upt"
    ]
    
    prev_ym_val =  df.loc[
        (df["period_year_month"] == prior_y_m) &
        (df["ntd_id"] == row["ntd_id"]) &
        (df["mode"] == row["mode"]) &
        (df["tos"] == row["tos"]), "upt"
    ]
    
    return (current_ym_val - prev_ym_val)

In [ ]:
test_df = df[:50].copy()

In [ ]:
test_df.columns

In [ ]:
year = 2023
month = "05"

current_y_m = f"{year}-{month}"
prior_y_m = f"{int(year)-1}-{month}"

display(
    current_y_m,
    prior_y_m
)

In [ ]:
test_df["test_col"] = test_df.apply(lambda row: test_difference_func(row, current_y_m="2023-05", prior_y_m="2022-05"), axis=1)

So im having trouble with coming up with a `change 1yr` column, now that the dataset is long (instead of the previous wide)

what i was going for was something like:
- adding a new column to the dataset called `change 1yr`, that takes the `UPT` value of the `ntd_id, mod, tos` for current `period_year_month` (ex. 2024-05) row, and subtracts the `UPT` value from the previous year's `period_year_month` (ex. 2024-05) row of the same `ntd_id, mod, tos`.

- i thought i had something with a .apply(lambda x:) method, but wasnt etting anywhere.

any suggestions on how i should approach this?


---

In [ ]:
# what is the most recent period_year_month? as of 7/2/2024\
# should expect to see values up tp May 2024
display(
    ca[ca["period_year_month"] == "2024-04"].shape,
    # as of 7/2, zero rows for May 2024. but expect ntd to update that data soon.
    # 7/15, still no values for May 2024
    ca[ca["period_year_month"] == "2024-05"].shape
)

# Tiffany method to get change 1yr working

[see link to her old GH sctipt](https://github.com/CityOfLosAngeles/covid19-indicators/blob/master/processing_utils/utils.py#L306-L315)

The .shift() method in Pandas is used to shift the values of a DataFrame or Series by a specified number of periods along the given axis.
When you use .shift(), the data in the DataFrame or Series is moved up or down, while the index/columns stay in place. can move the entire dataframe up/down/left/right, or just a row or column in any direction.

Per Tiffany
- "i think you could perhaps make this idea work. that snippet uses .shift to get you a previous observation and a post observation. in this case, it's the previous day and the day after. but for you, `you're looking at a previous year (same month)`."

```
sort_cols = ["county", "state", "fips", "date"]
group_cols = ["county", "state", "fips"]

df = df.assign(
        previous_day = (df.sort_values(sort_cols)
                        .groupby(group_cols)["new_cases"]
                        .apply(lambda x: x.shift(1))
                       ),
        post_day = (df.sort_values(sort_cols)
                    .groupby(group_cols)["new_cases"]
                    .apply(lambda x: x.shift(-1))
                   ),
    )
```

In [ ]:
# df of just riverside county transpo agency in select years
rcta = ca[(ca["ntd_id"].isin(["90218", "90006","90216"])) & (ca["period_year"].isin(["2024","2023", "2022", "2021"]))]


In [ ]:
rcta[["period_year","period_month"]] = rcta[["period_year","period_month"]].astype(int)

In [ ]:
display(
    rcta.columns,
    rcta.shape,
    rcta.dtypes,
    rcta["agency"].value_counts()
)

In [ ]:
# list of column names to sort the df by
# lets sort by mode, tos, period_year_month
sort_cols = ["mode", "tos", "period_year_month"]
group_cols = ["mode", "tos"]

current_y_m = f"{year}-{month}"
prior_y_m = f"{int(year)-1}-{month}"

In [ ]:
previous_day = (rcta.sort_values(sort_cols) #sort df by oldest date
                        .groupby(group_cols)["upt"] #returns only upt values
                        .apply(lambda x: x.shift(1)) #shifts upt values down by 1 step
                       )

next_day = (rcta.sort_values(sort_cols) #sort df by oldest date
                        .groupby(group_cols)["upt"] #returns only upt values
                        .apply(lambda x: x.shift(-1)) #shifts upt values up by 1 step
                       )
display(
    type(previous_day),
    rcta.sort_values(sort_cols).groupby(group_cols)["upt"].head(),
    previous_day.head(),
    next_day.head()

)
# result is a series of the "upt" col shifted down 1 place

In [ ]:
# put together, what happens?
# as is, new columns will be added to the DF, called prev_day and next_day. filled with values from upt shifted down one day and up one day respectivley. 

rcta.sort_values(sort_cols).groupby(group_cols)["upt"] = rcta.assign(
    previous_day = (rcta.sort_values(sort_cols) #sort df by oldest date
                        .groupby(group_cols)["upt"] #returns only upt values
                        .apply(lambda x: x.shift(1)) #shifts upt values down by 1 step
                       ),
    next_day = (rcta.sort_values(sort_cols) #sort df by oldest date
                        .groupby(group_cols)["upt"] #returns only upt values
                        .apply(lambda x: x.shift(-1)) #shifts upt values up by 1 step
                       ),

)
The above code shifts the UPT data down/up 1 step

In [ ]:
# How can i adapt this code to shift the just year?
# apply the shift to period_year?

sort_cols2 =  ["ntd_id","mode", "tos","period_month", "period_year"] # got the order correct with ["period_month", "period_year"]! sorted years with grouped months
group_cols2 = ["ntd_id","mode", "tos"]

rcta = rcta.assign(
    #this works! previous_y_m_upt shifted down 1 step as intended for each agency! each agency's first year_month got a NaN for previous_y_m_upt
    previous_y_m_upt = (rcta.sort_values(sort_cols2) #sort df by oldest date
                        .groupby(group_cols2)["upt"] #
                        .apply(lambda x: x.shift(1)) #shifts upt values down by 1 step
                       ),
    #THIS WORKS! the math works out!
    change_1yr = (rcta["upt"] - rcta["previous_y_m_upt"])
                  )


In [ ]:
display(
    #easy to see the differences when the DF is sorte the same way,
    rcta.sort_values(sort_cols2).head(),
    
    #does the math work out when the df is not sorted? YES IT IS!!
    rcta.sample(5)
)

# FINAL `get_percent_change_v2` function

initial code
```
def get_percent_change(
    df: pd.DataFrame, 
    current_col: str, 
    prior_col: str
) -> pd.DataFrame:
    
    df[f"pct_change_1yr_{current_col}"] = (
        (df[current_col] - df[prior_col])
        .divide(df[current_col])
        .round(4)
    )
    
    return df
```


In [5]:
def get_percent_change_v2(
    df: pd.DataFrame, 
) -> pd.DataFrame:
    """
    updated to work with the warehouse `dim_monthly_ntd_ridership_with_adjustments` long data format. 
    
    
    """
    df["pct_change_1yr"] = (
        (df["upt"] - df["previous_y_m_upt"])
        .divide(df["upt"])
        .round(4)
    )
    
    return df

In [ ]:
#it works!!
test_df = get_percent_change_v2(test_df)

test_df.head()

## ~How to deal with `inf` and `NaN`~ 
* `inf` when dividing by zero
* or `NaN` when 0 minus 0

SOLUTOIN! Already have `remove zero upt rows` function in notebooks



# FINAL `add_change_columns_v2` function

In [6]:
def add_change_columns_v2(df: pd.DataFrame) -> pd.DataFrame:
    """
    This function works with the warehouse `dim_monthly_ntd_ridership_with_adjustments` long data format.
    Sorts the df by ntd id, mode, tos, period month and period year. then adds 2 new columns, 1. previous year/month UPT and 2. UPT change 1yr.
    """

    sort_cols2 =  ["ntd_id","mode", "tos","period_month", "period_year"] # got the order correct with ["period_month", "period_year"]! sorted years with grouped months
    group_cols2 = ["ntd_id","mode", "tos"]
    
    df[["period_year","period_month"]] = df[["period_year","period_month"]].astype(int)

    df = df.assign(
        previous_y_m_upt = (df.sort_values(sort_cols2)
                        .groupby(group_cols2)["upt"] 
                        .apply(lambda x: x.shift(1))
                       )
    )

    df["change_1yr"] = (df["upt"] - df["previous_y_m_upt"])
    
    df = get_percent_change_v2(df)
    
    return df

In [ ]:
ca.shape

In [ ]:
#test to ensure `add_change_columns_v2` works
test_df = add_change_columns_v2(rcta)

big_test = add_change_columns_v2(ca)

really_big_test = add_change_columns_v2(full_upt)

In [ ]:
#it works!
display(
    test_df.shape,
    big_test.shape,
    really_big_test.shape,
    test_df.columns
)

display(
    rcta.shape,
    ca.shape,
    full_upt.shape
)

# ensuring the `.assign(NTD_MODES and NTD_TOS)` works.

In [ ]:
#it works
more_test = test_df.assign(
        Mode_full = test_df["mode"].map(NTD_MODES),
        TOS_full = test_df["tos"].map(NTD_TOS)
    )
more_test.columns

# FINAL produce_ntd_monthly_ridership_by_rtpa_v2 function

now that `add_change_col_v2` & `get_percent_change_v2` are working. lets put it all together

In [7]:
def produce_ntd_monthly_ridership_by_rtpa_v2(
    #df: pd.DataFrame,
    year: int,
    month: int
) -> pd.DataFrame:
    """
    This function works with the warehouse `dim_monthly_ntd_ridership_with_adjustments` long data format.
    """
    full_upt = (tbls.mart_ntd.dim_monthly_ntd_ridership_with_adjustments() >> collect()).rename(columns = {"mode_type_of_service_status": "Status"})
    
    #updating month & year to int is already in add_change_columns_v2. keeping here for now.
    #full_upt[["period_year","period_month"]] = full_upt[["period_year","period_month"]].astype(int)
    
    full_upt = full_upt[full_upt.agency.notna()].reset_index(drop=True)
    
    full_upt.to_parquet(
        f"{GCS_FILE_PATH}ntd_monthly_ridership_{year}_{month}.parquet"
    )
    
    ca = full_upt[(full_upt["uza_name"].str.contains(", CA")) & 
            (full_upt.agency.notna())].reset_index(drop=True)
    
    crosswalk = pd.read_csv(
        f"gs://calitp-analytics-data/data-analyses/ntd/ntd_id_rtpa_crosswalk.csv", 
        dtype = {"NTD ID": "str"}
    #have to rename NTD ID col to match the dim table
    ).rename(columns={"NTD ID": "ntd_id"})
    
    df = pd.merge(
        ca,
        # Merging on too many columns can create problems 
        # because csvs and dtypes aren't stable / consistent 
        # for NTD ID, Legacy NTD ID, and UZA
        crosswalk[["ntd_id", "RTPA"]],
        #change on = value to ntd_id
        on = "ntd_id",
        how = "left",
        indicator = True
    )
    
    print(df._merge.value_counts())
    
    if len(df[df._merge=="left_only"]) > 0:
        raise ValueError("There are unmerged rows to crosswalk")
    
    df = add_change_columns_v2(df)
    
    df = df.assign(
        Mode_full = df["mode"].map(NTD_MODES),
        TOS_full = df["tos"].map(NTD_TOS)
    )
    
    return df
    

# Full Run test of `produce_ntd_monthly...v2`

In [11]:
full_run_test = produce_ntd_monthly_ridership_by_rtpa_v2(YEAR, MONTH)
# it works!!
# also saving the temp GCS folder!

full_run_test.to_parquet(f"{GCS_FILE_PATH}ca_monthly_ridership_{YEAR}_{MONTH}.parquet")

both          92267
left_only         0
right_only        0
Name: _merge, dtype: int64


/tmp/ipykernel_304/3121106738.py:13: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  previous_y_m_upt = (df.sort_values(sort_cols2)


In [ ]:
display(
    full_run_test.columns,
    full_run_test.sample(3),
    full_run_test["reporter_type"].sort_values().unique()
)

# as of 7/22/2024. still only see '2024-04' as the latest date. need may 2024

# FINAL `save_rtpa_outputs_v2`

In [ ]:
# NOT FINAL
#def save_rtpa_outputs_v2(
    df: pd.DataFrame( 
    year: int, 
    month: str,
    upload_to_public: bool = False
):
    """
    Export a csv for each RTPA into a folder.
    Zip that folder. 
    Upload zipped file to GCS.
    """
    for i in df["RTPA"].unique():
        # Filename should be snakecase
        rtpa_snakecase = i.replace(' ', '_').lower()

        (df[df["RTPA"] == i]
         .sort_values("ntd_id")
         .drop(columns = "_merge")
         .to_csv(
            f"./{year}_{month}/{rtpa_snakecase}.csv",
            index = False)
        )
        
    shutil.make_archive(f"./{year}_{month}", "zip", f"{year}_{month}")
    print("Zipped folder")
    
    fs.upload(
        f"./{year}_{month}.zip", 
        f"{GCS_FILE_PATH}{year}_{month}.zip"
    )
    
    if upload_to_public:
        fs.upload(
            f"./{year}_{month}.zip",
            f"{PUBLIC_GCS}ntd_monthly_ridership/{year}_{month}.zip"
        )
    
    print("Uploaded to GCS")
    
    return

## Test `save_rtpa_outputs_v2`

initial code
```
def save_rtpa_outputs(
    df: pd.DataFrame, year: int, month: str,
    upload_to_public: bool = False
):
    """
    Export a csv for each RTPA into a folder.
    Zip that folder. 
    Upload zipped file to GCS.
    """
    for i in df.RTPA.unique():
        # Filename should be snakecase
        rtpa_snakecase = i.replace(' ', '_').lower()

        (df[df.RTPA == i]
         .sort_values("NTD ID")
         .drop(columns = "_merge")
         .to_csv(
            f"./{year}_{month}/{rtpa_snakecase}.csv",
            index = False)
        )
       
    # Zip this folder, and save zipped output to GCS
    shutil.make_archive(f"./{year}_{month}", "zip", f"{year}_{month}")
    print("Zipped folder")
    
    fs.upload(
        f"./{year}_{month}.zip", 
        f"{GCS_FILE_PATH}{year}_{month}.zip"
    )
    
    if upload_to_public:
        fs.upload(
            f"./{year}_{month}.zip",
            f"{PUBLIC_GCS}ntd_monthly_ridership/{year}_{month}.zip"
        )
    
    print("Uploaded to GCS")
    
    return
```

In [ ]:
# test to read in local excel sheet, and add it to another sheet
cover_sheet = pd.read_excel("./cover_sheet_template.xlsx")

In [ ]:
type(cover_sheet)

In [ ]:
#testing how to append a sheet to an existing workbook
#this works by itself
with pd.ExcelWriter("./2024_April/butte_county_association_of_governments.xlsx", mode ="a") as writer:
    cover_sheet.to_excel(writer, sheet_name = "1. Cover Sheet")

In [ ]:
# TEst to str replace and title case the columns name

test_df = full_run_test.copy()

test_df.columns


In [ ]:
test_df.columns = test_df.columns.str.replace("_", " ").str.strip().str.title()

#changes most of the columns but realized some of the columns 
# need to change seperately
list(test_df.columns)

In [ ]:
col_dict ={
    'Uace Cd': "UACE Code",
    'Dt': "Date",
    'Ntd Id': "NTD ID",
    'Tos': "Type of Service",
    'Legacy Ntd Id': "Legacy NTD ID",
    'Upt': "UPT",
    'Vrm': "VRM",
    'Vrh': "VRH",
    'Voms': "VOMS",
    'Rtpa': "RTPA",
    'Previous Y M Upt': "Previous Year/Month UPT",
    'Change 1Yr': "Change in 1 Year UPT",
    'Pct Change 1Yr': "Percent Change in 1 Year UPT",
    'Tos Full': "Type of Service Full Name"
}

test_df = test_df.rename(columns=col_dict)

list(test_df.columns)

In [9]:
# TEST VERSION

# updated to address feature requests raised by MTC and other RTPAs
# x save out as excel,
# x include a cover sheet/ READ ME 
# x unsnakecase the columns
# x use dict to rename remaining columns

def save_rtpa_outputs_v2(
    df: pd.DataFrame, 
    year: int, 
    month: str,
    upload_to_public: bool = False
):
    """
    Export a csv for each RTPA into a folder.
    Zip that folder. 
    Upload zipped file to GCS.
    """
    col_dict ={
    'Uace Cd': "UACE Code",
    'Dt': "Date",
    'Ntd Id': "NTD ID",
    'Tos': "Type of Service",
    'Legacy Ntd Id': "Legacy NTD ID",
    'Upt': "UPT",
    'Vrm': "VRM",
    'Vrh': "VRH",
    'Voms': "VOMS",
    'Rtpa': "RTPA",
    'Previous Y M Upt': "Previous Year/Month UPT",
    'Change 1Yr': "Change in 1 Year UPT",
    'Pct Change 1Yr': "Percent Change in 1 Year UPT",
    'Tos Full': "Type of Service Full Name"
}
    
    for i in df["RTPA"].unique():
        # Filename should be snakecase
        rtpa_snakecase = i.replace(' ', '_').lower()

        (df[df["RTPA"] == i]
         .sort_values("ntd_id")
         #got error from excel not recognizing timezone, made list to include dropping "ts" column
         .drop(columns = ["_merge","ts"])
         #cleaning column names
         .rename(columns=lambda x: x.replace("_"," ").title().strip())
         #rename columns
         .rename(columns=col_dict)
         #updated to `to_excel`, added sheet_name 
         .to_excel(
            f"./{year}_{month}/{rtpa_snakecase}.xlsx", sheet_name = "RTPA Ridership Data",
            index = False)
         
        )
        #insertng readme cover sheet, 
        cover_sheet = pd.read_excel("./cover_sheet_template.xlsx", index_col = "NTD Monthly Ridership by RTPA")
        with pd.ExcelWriter(f"./{year}_{month}/{rtpa_snakecase}.xlsx", mode ="a") as writer:
            cover_sheet.to_excel(writer, sheet_name = "READ ME")
        
    shutil.make_archive(f"./{year}_{month}", "zip", f"{year}_{month}")
    print("Zipped folder")
    
    fs.upload(
        f"./{year}_{month}.zip", 
        f"{GCS_FILE_PATH}{year}_{month}.zip"
    )
    
    if upload_to_public:
        fs.upload(
            f"./{year}_{month}.zip",
            f"{PUBLIC_GCS}ntd_monthly_ridership/{year}_{month}.zip"
        )
    
    print("Uploaded to GCS")
    
    return

In [12]:
# double check cwd
os.getcwd()

'/home/jovyan/data-analyses/ntd'

In [ ]:
# need to make directory for `save_rtpa_outputs_v2` to work

#confirmed this make a sub directory within /ntd

#ntd/2024_April
os.makedirs(f"./{YEAR}_{MONTH}/")

In [10]:
fs = gcsfs.GCSFileSystem()

#works!
# see csvs in the new 2024_April directory
# also uploated to temp CGS file

# takes a really long time since writing to excel is a lot slower. 
save_rtpa_outputs_v2(full_run_test,YEAR, MONTH, upload_to_public = False)

Zipped folder
Uploaded to GCS


# Test `remove_local_outputs` function

**NO CHANGES NEEDED**

In [14]:
def remove_local_outputs(year: int, month: str):
    shutil.rmtree(f"{year}_{month}/")
    os.remove(f"{year}_{month}.zip")

In [15]:
# yup this worked really fast, no changes are needed
remove_local_outputs(YEAR, MONTH)

# ---
# Things to Move Over

these are the final versions of the changes to move over 

## Functions to Move over

In [ ]:
def get_percent_change(
    df: pd.DataFrame, 
) -> pd.DataFrame:
    """
    updated to work with the warehouse `dim_monthly_ntd_ridership_with_adjustments` long data format. 
    
    """
    df["pct_change_1yr"] = (
        (df["upt"] - df["previous_y_m_upt"])
        .divide(df["upt"])
        .round(4)
    )
    
    return df

In [ ]:
def add_change_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    This function works with the warehouse `dim_monthly_ntd_ridership_with_adjustments` long data format.
    Sorts the df by ntd id, mode, tos, period month and period year. then adds 2 new columns, 1. previous year/month UPT and 2. UPT change 1yr.
    """

    sort_cols2 =  ["ntd_id","mode", "tos","period_month", "period_year"] # got the order correct with ["period_month", "period_year"]! sorted years with grouped months
    group_cols2 = ["ntd_id","mode", "tos"]
    
    df[["period_year","period_month"]] = df[["period_year","period_month"]].astype(int)

    df = df.assign(
        previous_y_m_upt = (df.sort_values(sort_cols2)
                        .groupby(group_cols2)["upt"] 
                        .apply(lambda x: x.shift(1))
                       )
    )

    df["change_1yr"] = (df["upt"] - df["previous_y_m_upt"])
    
    df = get_percent_change_v2(df)
    
    return df

In [ ]:
def produce_ntd_monthly_ridership_by_rtpa(
    year: int,
    month: int
) -> pd.DataFrame:
    """
    This function works with the warehouse `dim_monthly_ntd_ridership_with_adjustments` long data format.
    """
    full_upt = (tbls.mart_ntd.dim_monthly_ntd_ridership_with_adjustments() >> collect()).rename(columns = {"mode_type_of_service_status": "Status"})
    
    full_upt = full_upt[full_upt.agency.notna()].reset_index(drop=True)
    
    full_upt.to_parquet(
        f"{GCS_FILE_PATH}ntd_monthly_ridership_{year}_{month}.parquet"
    )
    
    ca = full_upt[(full_upt["uza_name"].str.contains(", CA")) & 
            (full_upt.agency.notna())].reset_index(drop=True)
    
    crosswalk = pd.read_csv(
        f"gs://calitp-analytics-data/data-analyses/ntd/ntd_id_rtpa_crosswalk.csv", 
        dtype = {"NTD ID": "str"}
    #have to rename NTD ID col to match the dim table
    ).rename(columns={"NTD ID": "ntd_id"})
    
    df = pd.merge(
        ca,
        # Merging on too many columns can create problems 
        # because csvs and dtypes aren't stable / consistent 
        # for NTD ID, Legacy NTD ID, and UZA
        crosswalk[["ntd_id", "RTPA"]],
        on = "ntd_id",
        how = "left",
        indicator = True
    )
    
    print(df._merge.value_counts())
    
    if len(df[df._merge=="left_only"]) > 0:
        raise ValueError("There are unmerged rows to crosswalk")
    
    df = add_change_columns_v2(df)
    
    df = df.assign(
        Mode_full = df["mode"].map(NTD_MODES),
        TOS_full = df["tos"].map(NTD_TOS)
    )
    
    return df

In [ ]:
def save_rtpa_outputs(
    df: pd.DataFrame, 
    year: int, 
    month: str,
    upload_to_public: bool = False
):
    """
    Export a csv for each RTPA into a folder.
    Zip that folder. 
    Upload zipped file to GCS.
    """
    col_dict ={
    'Uace Cd': "UACE Code",
    'Dt': "Date",
    'Ntd Id': "NTD ID",
    'Tos': "Type of Service",
    'Legacy Ntd Id': "Legacy NTD ID",
    'Upt': "UPT",
    'Vrm': "VRM",
    'Vrh': "VRH",
    'Voms': "VOMS",
    'Rtpa': "RTPA",
    'Previous Y M Upt': "Previous Year/Month UPT",
    'Change 1Yr': "Change in 1 Year UPT",
    'Pct Change 1Yr': "Percent Change in 1 Year UPT",
    'Tos Full': "Type of Service Full Name"
}
    
    for i in df["RTPA"].unique():
        # Filename should be snakecase
        rtpa_snakecase = i.replace(' ', '_').lower()

        (df[df["RTPA"] == i]
         .sort_values("ntd_id")
         #got error from excel not recognizing timezone, made list to include dropping "ts" column
         .drop(columns = ["_merge","ts"])
         #cleaning column names
         .rename(columns=lambda x: x.replace("_"," ").title().strip())
         #rename columns
         .rename(columns=col_dict)
         #updated to `to_excel`, added sheet_name 
         .to_excel(
            f"./{year}_{month}/{rtpa_snakecase}.xlsx", sheet_name = "RTPA Ridership Data",
            index = False)
         
        )
        #insertng readme cover sheet, 
        cover_sheet = pd.read_excel("./cover_sheet_template.xlsx", index_col = "NTD Monthly Ridership by RTPA")
        with pd.ExcelWriter(f"./{year}_{month}/{rtpa_snakecase}.xlsx", mode ="a") as writer:
            cover_sheet.to_excel(writer, sheet_name = "READ ME")
        
    shutil.make_archive(f"./{year}_{month}", "zip", f"{year}_{month}")
    print("Zipped folder")
    
    fs.upload(
        f"./{year}_{month}.zip", 
        f"{GCS_FILE_PATH}{year}_{month}.zip"
    )
    
    if upload_to_public:
        fs.upload(
            f"./{year}_{month}.zip",
            f"{PUBLIC_GCS}ntd_monthly_ridership/{year}_{month}.zip"
        )
    
    print("Uploaded to GCS")
    
    return

## Misc. Script changes 

## Notebook Functions and changes to move over

In [ ]:
MIN_YEAR = 2018

#updated to filter for period_year greater than 2018
df = pd.read_parquet(
    f"{GCS_FILE_PATH}ca_monthly_ridership_{YEAR}_{MONTH}.parquet",
    filters = [[("RTPA", "==", rtpa), ("period_year", ">=", MIN_YEAR)]]
).drop(
    #updated columns names to match new df
    columns = ["mode", "tos"]
).rename(columns = {"Mode_full": "Mode", "TOS_full": "TOS"})

In [ ]:
def sum_by_group(
    df: pd.DataFrame,
    group_cols: list) -> pd.DataFrame:
    """
    since data is now long to begin with, this replaces old sum_by_group, make_long and assemble_long_df functions.
    """
    from TEST_monthly_ridership_by_rtpa import get_percent_change
    grouped_df = df.groupby(group_cols+
                             ['period_year',
                             'period_month',
                             'period_year_month']
                           ).agg({
        "upt":"sum",
        "previous_y_m_upt":"sum",
        "change_1yr":"sum"
    }
    ).reset_index()
    
    #get %change back
    grouped_df = get_percent_change(grouped_df)
    
    #decimal to whole number
    grouped_df["pct_change_1yr"] = grouped_df["pct_change_1yr"]*100
    
    return grouped_df

In [ ]:
agency_cols = ["ntd_id", "agency", "RTPA"]
mode_cols = ["Mode", "RTPA"]
tos_cols = ["TOS", "RTPA"]

by_agency_long = sum_by_group(df, agency_cols)
by_mode_long = sum_by_group(df, mode_cols)
by_tos_long = sum_by_group(df, tos_cols)

# zero upt still works!
by_agency_long_no_zero_upt = remove_zero_upt_rows(by_agency_long)

In [ ]:
def make_line_chart(
    df: pd.DataFrame, 
    y_col: str,
    color_col: str,
) -> alt.Chart:
    df = df[df[y_col] > 0].dropna(subset = y_col)

    x_label = [i for i in df.period_year_month.unique() if 
               any(substring in i for substring in 
                   ["-01", "-06"])
              ]    
    chart = (alt.Chart(df)
         .mark_line()
         .encode(
             x = alt.X("period_year_month:O", 
                       axis=alt.Axis(values = x_label), 
                       title = "Date"
                      ),
             y = alt.Y(y_col, title = labeling(y_col)),
             color = alt.Color(color_col, title = "", 
                               scale = alt.Scale(
                                   range = cp.CALITP_CATEGORY_BRIGHT_COLORS + 
                                   cp.CALITP_CATEGORY_BOLD_COLORS
                               )),
             tooltip = ["period_year_month", y_col, color_col, "RTPA"]
         ).properties(width = WIDTH, height = HEIGHT)
         .facet(color_col, columns=2, title = "")
         .resolve_scale(y="independent")
    ).properties(
        title = f"{labeling(y_col)} by {labeling(color_col)}"
    ).interactive()
        
    return chart

In [ ]:
def make_bar_chart(
    df: pd.DataFrame, 
    y_col: str,
    color_col: str,
) -> alt.Chart:
    
    def short_label(word):
        shorten_dict = {
            "change_1yr": "Change",
            "pct_change_1yr":"Change",
        }
        return shorten_dict[word]
    
    # For change column, we are missing everything prior to 2023 
    #df = df.dropna(subset = y_col)
    
    #need flag for y_col >,<, 0, missing?
    #count function to how many agencies fall in those categories, then look at those agencies
    #present table
    
    x_label = [i for i in df.period_year_month.unique() if 
               any(substring in i for substring in 
                   ["-01", "-03", "-06", "-09"])
              ]
    
    chart = (alt.Chart(df)
         .mark_bar()
         .encode(
             x = alt.X("period_year_month:O", 
                       axis=alt.Axis(values = x_label), 
                       title = "Date"
                      ),
             y = alt.Y(y_col, title = short_label(y_col)),
             color = alt.Color(color_col, title = "", 
                              scale = alt.Scale(
                                   range = cp.CALITP_CATEGORY_BRIGHT_COLORS + 
                                   cp.CALITP_CATEGORY_BOLD_COLORS
                              )),
             tooltip = ["period_year_month", y_col, color_col, "RTPA"]
         ).properties(width = WIDTH, height = HEIGHT)
         .facet(color_col, columns=2, title = "")
         .resolve_scale(x="shared", 
                        y="independent")
    ).properties(
        title = f"{labeling(y_col)} by {labeling(color_col)}"
    ).interactive()
    
    return chart